In [4]:
import pandas as pd
import pickle
import lightgbm as lgb
import glob
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed
import time
from sklearn.metrics import log_loss, accuracy_score, f1_score, plot_roc_curve
import datetime
from sklearn.metrics import classification_report, confusion_matrix
import optuna
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import log_loss, accuracy_score, f1_score, plot_roc_curve
from sklearn.linear_model import LogisticRegression


In [2]:
print('========LOADING DATA========')
df_500 = pd.read_csv('pct1_cal/modified_alter_alphas_066_labels_500.csv')
print(df_500.shape)
print('========COMPLETE LOADING DATA========')

========LOADING DATA========
(569537, 293)
========COMPLETE LOADING DATA========


In [17]:
percent_samples = [0.5, 0.4, 0.3, 0.2, 0.1] # 训练集样本量 0.5表示前后50%都选 即全部数据都选；0.3表示取涨幅前后30% 共60% 的样本量 
percent_sample = percent_samples[2]

In [22]:
# 采样比例
if percent_sample==0.5:
    qcut_q = [0, 0.5, 1]
    qcut_labels = [0, 1]
else:
    qcut_q = [0, percent_sample, 1-percent_sample, 1]
    qcut_labels = [0, -1, 1]

In [23]:
df_500['openclose_pct1_label'] = df_500.groupby('tradeDate').openclose_pct1_rank.apply(lambda x: pd.qcut(x, q=qcut_q, labels=qcut_labels, duplicates='drop'))

ValueError: Bin labels must be one fewer than the number of bin edges

In [ ]:
df_500 = df_500[(df_500['openclose_pct1_label']==0) | (df_500['openclose_pct1_label']==1)]

In [24]:
df_500

,ticker,tradeDate,minute_new_FACTORS_CDPDP,minute_new_FACTORS_CDPP,minute_new_FACTORS_Down_beta,minute_new_FACTORS_ILLIQ,minute_new_FACTORS_IMI,minute_new_FACTORS_PVC,minute_new_FACTORS_TMA_turn,minute_new_FACTORS_TSMOM,...,alpha_27,alpha_11,alpha_42,alpha_12,alpha_1,alpha_23,alpha_5,alpha_9,alpha_31,openclose_pct1_rank
0,9,2017-01-03,-1.47100,2.125000,0.0000,-0.694300,0.000000,2.256000,0.6616,-0.2969,...,0.00000,0.0000,0.00000,0.00000,0.000,0.0000,0.0000,0.0,0.0000,0.79600
1,9,2017-01-04,-1.40400,1.884000,0.0000,-0.682000,0.000000,2.074000,0.9795,-0.8984,...,0.00000,0.0000,0.00000,0.00000,0.000,0.0000,0.0000,0.0,0.0000,0.18620
2,9,2017-01-05,-0.00638,1.806000,0.0000,-0.708000,0.000000,2.023000,1.0360,-1.1230,...,0.00000,0.0000,0.00000,0.00000,0.000,0.0000,0.0000,0.0,0.0000,0.63600
3,9,2017-01-06,-0.01643,1.675000,0.0000,-0.868000,0.000000,1.468000,1.0280,-0.8257,...,0.00000,0.0000,0.00000,0.00000,0.000,0.0000,0.0000,0.0,0.0000,0.68500
4,9,2017-01-09,-0.01038,1.354000,0.0000,-0.914600,0.000000,1.424000,1.0520,-0.8170,...,0.00000,0.0000,0.00000,0.00000,0.000,0.0000,0.0000,0.0,0.0000,0.84900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569532,689009,2022-03-28,0.00000,-0.043100,0.0841,-0.005463,-0.592000,-0.072750,0.0000,0.7573,...,0.24490,0.5015,-0.04517,0.00635,1.895,2.7420,-0.2764,0.0,0.1385,0.90870
569533,689009,2022-03-29,0.00000,0.131500,0.0883,-0.005413,-0.003313,0.048680,0.0000,0.3320,...,-0.04490,0.5640,-0.04993,1.19700,1.558,-2.0760,-0.4504,0.0,0.2303,0.02888
569534,689009,2022-03-30,0.00000,0.015480,0.1278,-0.012070,-0.546000,0.088750,0.0000,-0.8220,...,-0.04490,0.6514,-0.04703,-1.55500,-1.620,0.2030,0.0000,0.0,0.2053,0.34350
569535,689009,2022-03-31,0.00000,-0.019520,0.5350,-0.015750,-0.527000,-0.044400,0.0000,-0.9720,...,-0.04495,0.6450,-0.04620,1.47800,-1.434,2.0600,0.4358,0.0,0.2075,0.27600


In [ ]:
df_500.openclose_pct1_label.value_counts()

0     34767
1     34658
-1        0
Name: openclose_pct1_label, dtype: int64

In [43]:
dates = df_500.tradeDate.sort_values().unique()
epoch_ts = list(dates)

f_index = ['ticker', 'tradeDate']
f_x = pickle.load(open("pct1_cal/f_x_066", "rb"))
label_list = ['PCT5_rank', 'PCT2_rank', 'openclose_pct1_rank', 'askbid_pct1_rank']
f_y = 'openclose_pct1_label' #label_list[2]
#model_list = ['RidgeR', 'DecisionTreeR', 'XGBoostR', 'LGBMRegressor']
model_name = 'LGBMClassifier'#model_list[3]

print('======== LEN_TRAIN {} ========'.format(f_y))
target_types = ['r', 'c'] # 分类问题还是回归问题 r 回归问题 c 分类问题
target_type = target_types[1]

result_name = '500_Alter_066_full_{}-{}-{}'.format(model_name, f_y, target_type)
print(result_name)

update = 22 # 训练长度：22天
train_si = epoch_ts.index('2017-01-03') # included. '2017-01-03'
train_ei = epoch_ts.index('2019-01-02') # excluded. '2018-12-28'
test_si = epoch_ts.index('2019-01-02') # included. '2019-01-02'
test_ei = epoch_ts.index('2019-02-01') # excluded. '2019-01-31'
test_fi = len(epoch_ts) - 1 # excluded.

# number of epochs，循环次数
num_epoch = round((test_fi - test_ei) / 22)
epoch_range = range(0, num_epoch + 1)
#epoch_range = range(0, 1)

start = time.time()
df_result_all = pd.DataFrame()
for epoch in epoch_range:
    print('----- EPOCH {}------'.format(epoch))
    update_n = epoch * update
    # get a list of train dates
    epoch_t_train = epoch_ts[train_si + update_n : train_ei + update_n]
    # get a list of test dates
    epoch_t_test = epoch_ts[test_si + update_n : test_ei + update_n]
    df_train = df_500[df_500.tradeDate.apply(lambda x: x in epoch_t_train)].reset_index(drop=True)
    df_test = df_500[df_500.tradeDate.apply(lambda x: x in epoch_t_test)].reset_index(drop=True)
    print('预测时间：', epoch_t_test)
    print('数据大小：', df_train.shape, df_test.shape)

    x_train = df_train[f_x].values
    x_test = df_test[f_x].values

    # 获得y
    y_train = df_train[f_y].copy()
    y_test = df_test[f_y].copy()
    print('y：', y_train.shape, y_test.shape)

    model_1 = LogisticRegression(C = 1, solver='liblinear', max_iter = 300)
    model_1.fit(x_train, y_train)

    #model_2 = lgb.LGBMClassifier(max_depth=5, num_leaves=10, learning_rate=0.05, n_estimators=200)
    #model_2.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_test, y_test)], eval_metric='logloss')
    y_pred = model_1.predict(x_test)

    # 获得结果
    print('get result')
    df_result = df_test[f_index].copy()
    df_result['y'] = y_test
    df_result['y_pred'] = y_pred
    df_result_all = df_result_all.append(df_result)

print(f'耗时:{time.time() - start}') 
print('sort values')
df_result_all = df_result_all.sort_values(by=['ticker', 'tradeDate']).reset_index(drop=True)
#today = (datetime.datetime.now()).strftime("%Y-%m-%d:%H:%M:%S")
#df_result_all.to_csv('pct1_cal/ZZ500/{}'+today+'.csv'.format(result_name), index=False)
score = accuracy_score(df_result_all['y'], df_result_all['y_pred'])
print('======== COMPLETE {} {} ========'.format(model_name, score))

======== LEN_TRAIN openclose_pct1_label ========
500_Alter_066_full_LGBMClassifier-openclose_pct1_label-c
----- EPOCH 0------
预测时间： ['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11', '2019-01-14', '2019-01-15', '2019-01-16', '2019-01-17', '2019-01-18', '2019-01-21', '2019-01-22', '2019-01-23', '2019-01-24', '2019-01-25', '2019-01-28', '2019-01-29', '2019-01-30', '2019-01-31']
数据大小： (24008, 294) (1180, 294)
y： (24008,) (1180,)
get result
----- EPOCH 1------
预测时间： ['2019-02-01', '2019-02-11', '2019-02-12', '2019-02-13', '2019-02-14', '2019-02-15', '2019-02-18', '2019-02-19', '2019-02-20', '2019-02-21', '2019-02-22', '2019-02-25', '2019-02-26', '2019-02-27', '2019-02-28', '2019-03-01', '2019-03-04', '2019-03-05', '2019-03-06', '2019-03-07', '2019-03-08', '2019-03-11']
数据大小： (24176, 294) (1188, 294)
y： (24176,) (1188,)
get result
----- EPOCH 2------
预测时间： ['2019-03-12', '2019-03-13', '2019-03-14', '2019-03-15', '2019-03-18', '20

In [44]:
df_result_all.y.value_counts()

0     22739
1     22678
-1        0
Name: y, dtype: int64

In [45]:
df_result_all.y_pred.value_counts()

0    23008
1    22409
Name: y_pred, dtype: int64

In [46]:
print( classification_report(df_result_all['y'], df_result_all['y_pred']) )
print( confusion_matrix(df_result_all['y'], df_result_all['y_pred']) )
plot_roc_curve( model_1, x_test, y_test )

              precision    recall  f1-score   support

           0       0.53      0.54      0.54     22739
           1       0.53      0.53      0.53     22678

    accuracy                           0.53     45417
   macro avg       0.53      0.53      0.53     45417
weighted avg       0.53      0.53      0.53     45417

[[12277 10462]
 [10731 11947]]


In [ ]:
dates = df_500.tradeDate.sort_values().unique()
epoch_ts = list(dates)

f_index = ['ticker', 'tradeDate']
f_x = pickle.load(open("pct1_cal/f_x_066", "rb"))
f_y = 'openclose_pct1_label' #'openclose_pct1_rank'

update = 22 # 训练长度：22天
train_si = epoch_ts.index('2017-01-03') # included. '2017-01-03'
train_ei = epoch_ts.index('2019-01-02') # excluded. '2018-12-28'
test_si = epoch_ts.index('2019-01-02') # included. '2019-01-02'
test_ei = epoch_ts.index('2019-02-01') # excluded. '2019-01-31'
test_fi = len(epoch_ts) - 1 # excluded.

# number of epochs，循环次数
num_epoch = round((test_fi - test_ei) / 22)
#epoch_range = range(0, num_epoch + 1)
#epoch_range = range(0, 1)

start = time.time()
df_result_all = pd.DataFrame()

epoch_t_train = epoch_ts[train_si: train_ei]
epoch_t_test = epoch_ts[test_si: test_ei]
df_train = df_500[df_500.tradeDate.apply(lambda x: x in epoch_t_train)].reset_index(drop=True)
df_test = df_500[df_500.tradeDate.apply(lambda x: x in epoch_t_test)].reset_index(drop=True)
print('预测时间：', epoch_t_test)
print('数据大小：', df_train.shape, df_test.shape)

x_train = df_train[f_x].values
x_test = df_test[f_x].values
print('处理后x：', x_train.shape, x_test.shape)

# 获得y
y_train = df_train[f_y].copy()
y_test = df_test[f_y].copy()
print('处理后y：', y_train.shape, y_test.shape)

预测时间： ['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11', '2019-01-14', '2019-01-15', '2019-01-16', '2019-01-17', '2019-01-18', '2019-01-21', '2019-01-22', '2019-01-23', '2019-01-24', '2019-01-25', '2019-01-28', '2019-01-29', '2019-01-30', '2019-01-31']
数据大小： (24008, 294) (1180, 294)
处理后x： (24008, 290) (1180, 290)
处理后y： (24008,) (1180,)


In [ ]:
model_1 = LogisticRegression(C = 1, solver='liblinear', max_iter = 300)
model_1.fit(x_train, y_train)
y_pred_1 = model_1.predict(x_test)
print( classification_report(y_test, y_pred_1))
print( confusion_matrix(y_test, y_pred_1))
#plot_roc_curve( model_1, x_test, y_test )

              precision    recall  f1-score   support

           0       0.54      0.56      0.55       591
           1       0.54      0.51      0.53       589

    accuracy                           0.54      1180
   macro avg       0.54      0.54      0.54      1180
weighted avg       0.54      0.54      0.54      1180

[[333 258]
 [287 302]]


In [112]:
y_score_train = model_1.predict_proba(x_train)[:,1]
print(y_score_train)
thres = 0.3
prediction_high_recall_train = (y_score_train > thres).astype(int)
print( classification_report(y_train, prediction_high_recall_train))
print( confusion_matrix(y_train, prediction_high_recall_train))
meta_x_train = np.hstack((prediction_high_recall_train.reshape(-1,1), x_train))
meta_y_train = y_train.astype(int) #& prediction_high_recall_train

[0.60816699 0.54379571 0.67394488 ... 0.58332873 0.56612338 0.46547936]
              precision    recall  f1-score   support

           0       0.71      0.03      0.07     12028
           1       0.50      0.99      0.67     11980

    accuracy                           0.51     24008
   macro avg       0.61      0.51      0.37     24008
weighted avg       0.61      0.51      0.37     24008

[[  416 11612]
 [  172 11808]]


In [113]:
y_score_test = model_1.predict_proba(x_test)[:,1]
print(y_score_test)
thres = 0.3
prediction_high_recall_test = (y_score_test > thres).astype(int)
print( classification_report(y_test, prediction_high_recall_test ))
print( confusion_matrix(y_test, prediction_high_recall_test ))
meta_x_test = np.hstack((prediction_high_recall_test .reshape(-1,1), x_test))
meta_y_test = y_test.astype(int) #& prediction_high_recall_test 

[0.58391511 0.64599229 0.52987718 ... 0.48654045 0.48755298 0.43106422]
              precision    recall  f1-score   support

           0       0.71      0.05      0.09       591
           1       0.51      0.98      0.67       589

    accuracy                           0.51      1180
   macro avg       0.61      0.52      0.38      1180
weighted avg       0.61      0.51      0.38      1180

[[ 30 561]
 [ 12 577]]


In [114]:
model_3 = LogisticRegression(C = 1, solver='liblinear', max_iter = 300)
model_3.fit(meta_x_train, meta_y_train)
y_pred_3 = model_3.predict(meta_x_test)
final_prediction = y_pred_3 & prediction_high_recall_test
print( classification_report(meta_y_test, final_prediction))
print( confusion_matrix(meta_y_test, final_prediction))
print(accuracy_score(meta_y_test, final_prediction))
#pd.DataFrame(final_prediction).value_counts()

              precision    recall  f1-score   support

           0       0.54      0.57      0.55       591
           1       0.54      0.51      0.53       589

    accuracy                           0.54      1180
   macro avg       0.54      0.54      0.54      1180
weighted avg       0.54      0.54      0.54      1180

[[335 256]
 [287 302]]
0.5398305084745763


In [94]:
model_2 = lgb.LGBMClassifier(max_depth=5, num_leaves=10, learning_rate=0.09, n_estimators=100)
model_2.fit(meta_x_train, meta_y_train, eval_set=[(meta_x_train, meta_y_train), (meta_x_test, meta_y_test)], eval_metric='logloss')
meta_y_pred = model_2.predict(meta_x_test)
final_prediction = meta_y_pred & prediction_high_recall_test
print( classification_report(y_test, final_prediction))
print( confusion_matrix(y_test, final_prediction))
print(accuracy_score(meta_y_test, final_prediction))
#pd.DataFrame(final_prediction).value_counts()

[1]	training's binary_logloss: 0.566613	valid_1's binary_logloss: 0.53846
[2]	training's binary_logloss: 0.536407	valid_1's binary_logloss: 0.511297
[3]	training's binary_logloss: 0.510804	valid_1's binary_logloss: 0.487847
[4]	training's binary_logloss: 0.489056	valid_1's binary_logloss: 0.468153
[5]	training's binary_logloss: 0.470373	valid_1's binary_logloss: 0.451497
[6]	training's binary_logloss: 0.454258	valid_1's binary_logloss: 0.43745
[7]	training's binary_logloss: 0.440255	valid_1's binary_logloss: 0.42477
[8]	training's binary_logloss: 0.427974	valid_1's binary_logloss: 0.413982
[9]	training's binary_logloss: 0.41724	valid_1's binary_logloss: 0.404434
[10]	training's binary_logloss: 0.407797	valid_1's binary_logloss: 0.396274
[11]	training's binary_logloss: 0.39944	valid_1's binary_logloss: 0.388988
[12]	training's binary_logloss: 0.392008	valid_1's binary_logloss: 0.382932
[13]	training's binary_logloss: 0.385478	valid_1's binary_logloss: 0.377396
[14]	training's binary_log

In [41]:
from sklearn.model_selection import GridSearchCV
def build_model_1(x_train, y_train):
    
    logitreg_parameters = {'C': np.power(10.0, np.arange(-9, 1))}
    
    model_1 = LogisticRegression(solver='liblinear', max_iter = 300)
    logitreg_grid = GridSearchCV(model_1, param_grid = logitreg_parameters,scoring = 'f1', n_jobs = 1, cv=5)          
                             
    logitreg_grid.fit(x_train, y_train)
    
    return logitreg_grid
model_1 = build_model_1(x_train, y_train)

In [42]:
model_1.best_params_

{'C': 1.0}

In [38]:
def objective(trial):
    params = {
        'C': trial.suggest_float('C', np.power(10.0, -9), np.power(10.0, 1)),
        #'solver' : trial.suggest_categorical('solver', ['lbfgs', 'liblinear'])
        #"n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        #"learning_rate": trial.suggest_float("learning_rate", 0.01, 0.15),
    }
    # 'n_estimators': [20], 'max_depth': [3], 'max_features': [10], 'subsample': [1.0]},
    # 'splitter': ['random'], 'criterion': ['friedman_mse'], 'max_depth': [6], 
    # 'min_samples_leaf': [41], 'min_impurity_decrease': [0.5]
    cv = TimeSeriesSplit(n_splits=3)

    cv_scores = np.empty(3)

    # LGBM
    for idx, (train_idx, test_idx) in enumerate(cv.split(df_train[f_x], df_train[f_y])):
        x_cv_train, x_cv_test = df_train[f_x].iloc[train_idx], df_train[f_x].iloc[test_idx]
        y_cv_train, y_cv_test = df_train[f_y][train_idx], df_train[f_y][test_idx]

        model = LogisticRegression(solver= 'liblinear', **params)
        model.fit(x_cv_train, y_cv_train)
        #model = lgb.LGBMClassifier(max_depth=5, num_leaves=10, **params)
        #model.fit(x_cv_train, y_cv_train, eval_set=[(x_cv_train, y_cv_train), \
        #    (x_cv_test, y_cv_test)], eval_metric='logloss')
        y_pred = model.predict(x_cv_test)

            # 获得结果
        #df_result = pd.DataFrame({'y' : y_test, 'y_pred': y_pred})
        #IC = df_result[['y', 'y_pred']].corr().iloc[0,1]
        cv_scores[idx] = accuracy_score(y_cv_test, y_pred)
    return np.mean(cv_scores)

In [39]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

[I 2022-06-28 17:30:09,414] A new study created in memory with name: no-name-8849177e-a2d5-4f41-96eb-f41b82db29b4
/workspace1/liufengyuan/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/workspace1/liufengyuan/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html

In [ ]:
model = lgb.LGBMClassifier(max_depth=5, num_leaves=10, learning_rate=0.09, n_estimators=100)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), \
    (x_test, y_test)], eval_metric='logloss')
y_pred = model.predict(x_test)

    # 获得结果
#df_result = pd.DataFrame({'y' : y_test, 'y_pred': y_pred})
#IC = df_result[['y', 'y_pred']].corr().iloc[0,1]
accuracy_score(y_test, y_pred)
# 200: 0.89985, 0.09

In [ ]:
df_result = pd.DataFrame(index=df_train.index)
df_result['y'] = y_test
df_result['y_pred'] = y_pred